# Kernel Methods - Homework

__Métodos Intensivos de Computación Estadística__

Juan Sebastián Corredor Rodriguez - jucorredorr@unal.edu.co

See my [Github Account](https://github.com/juanse1608) to know more about me and my projects.

In [20]:
#Loading the required libraries
#install.packages('caret')
#install.packages('gamlss')
#install.packages('e1071')
#devtools::install_github("r-lib/rlang", build_vignettes = TRUE)
library(gamlss)
library(caret)
library(e1071)

The main idea of this script is to see a to classify the breast cancer diagnosis using GAMLSS models

---

In [3]:
#Let's plot the working directory
setwd('../../Datasets/')

In [4]:
#Let's load the names and the data
names = read.table('wpbc_names.dat', stringsAsFactors = F)
dta = read.table('breast_data.dat', sep = ',')

In [5]:
colnames(dta) = unique(names)[1:32,]

In [6]:
#Let's assign
colnames(dta)  = unique(names[,1])
print(head(dta))

ERROR: Error in names(x) <- value: el atributo 'names' [33] debe tener la misma longitud que el vector [32]


In [7]:
#Remove ids
dta$id= NULL

In [8]:
#Let's see the distribution of the diagnosis 
table(dta$diagnosis)
#Looks not at all inflated


  B   M 
357 212 

In [9]:
trains = sample(1:nrow(dta), size = ceiling(0.8*nrow(dta)), replace = FALSE)
dta_train = dta[trains,]
dta_test = dta_train[-trains,]

In [10]:
control = gamlss.control(trace = F)
m_binomial = gamlss(diagnosis ~ ., data = dta_train, family = BI, control = control)
print(paste('The AIC is', GAIC(m_binomial)))

[1] "The AIC is 999.134988117046"


In [11]:
options(warn=-1)
summary(m_binomial)

******************************************************************
Family:  c("BI", "Binomial") 

Call:  gamlss(formula = diagnosis ~ ., family = BI, data = dta_train,  
    control = control) 

Fitting method: RS() 

------------------------------------------------------------------
Mu link function:  logit
Mu Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)            -4.855e+10  2.002e+05 -242576   <2e-16 ***
radius_mean             7.250e+09  7.934e+04   91377   <2e-16 ***
texture_mean           -6.764e+07  3.860e+03  -17524   <2e-16 ***
perimeter_mean         -8.045e+08  1.160e+04  -69365   <2e-16 ***
area_mean              -1.395e+07  2.599e+02  -53667   <2e-16 ***
smoothness_mean         9.141e+09  9.744e+05    9381   <2e-16 ***
compactness_mean       -5.793e+09  6.335e+05   -9145   <2e-16 ***
concavity_mean          2.082e+10  4.956e+05   42000   <2e-16 ***
concave                -8.856e+09  9.640e+05   -9187   <2e-16 ***
points_mean  

We can see with this model that all the coefficients are significant.

---

Now let's see how it predicts data

In [12]:
predictions = round(predict(object = m_binomial, newdata = dta_test[,-1], type='response'))
expected = as.numeric(dta_test$diagnosis == 'M')

In [17]:
#Let's take a look at the mse 
mse = mean(abs(predictions - expected))
print(mse)

[1] 0.01086957


It is pretty lower, the model it is almost perfect. GOOD! 

In [21]:
#The confusion matrix
confusionMatrix(data = as.factor(predictions), reference = as.factor(expected))

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 52  0
         1  1 39
                                          
               Accuracy : 0.9891          
                 95% CI : (0.9409, 0.9997)
    No Information Rate : 0.5761          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9778          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9811          
            Specificity : 1.0000          
         Pos Pred Value : 1.0000          
         Neg Pred Value : 0.9750          
             Prevalence : 0.5761          
         Detection Rate : 0.5652          
   Detection Prevalence : 0.5652          
      Balanced Accuracy : 0.9906          
                                          
       'Positive' Class : 0               
                                    

We can see very nice results, with an accuracy of 98.91%.